In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#read data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test  = mnist.test.images
y_test  = mnist.test.labels

#set parameters
n_features = x_train.shape[1]
n_labels = y_train.shape[1]

#set input/output
sess = tf.InteractiveSession()
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32, shape = [None, n_features])
with tf.name_scope('Labels'):
    y = tf.placeholder(tf.float32, shape = [None, n_labels])

#def varible
def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

#def conv and maxpool
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides = [1,1,1,1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

#set layer
#first
with tf.name_scope('FirstConvolutionLayer'):
    wei_conv_1 = weight_variable([5,5,1,32])
    bia_conv_1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,28,28,1])
#tf.reshape(x, [-1???,28,28,1])
    conv_1 = tf.nn.relu(conv2d(x_image, wei_conv_1) + bia_conv_1)
    max_pool_1 = max_pool_2x2(conv_1)

#sec
with tf.name_scope('SecConvLayer'):
    wei_conv_2 = weight_variable([5,5,32,64])
    bia_conv_2 = bias_variable([64])
    conv_2 = tf.nn.relu(conv2d(max_pool_1, wei_conv_2) + bia_conv_2)
    max_pool_2 = max_pool_2x2(conv_2)
    
#full connection
with tf.name_scope('FullConnectionLayer'):
    wei_conv_3 = weight_variable([7 * 7 *64, 1024])
    bia_conv_3 = bias_variable([1024])
    max_pool_2_flat = tf.reshape(max_pool_2, [-1, 7 * 7 * 64])
    fcon_1 = tf.nn.relu(tf.matmul(max_pool_2_flat, wei_conv_3) + bia_conv_3)

#dropout fun
with tf.name_scope('DropFun'):
    keep_prob = tf.placeholder(tf.float32)
    fcon_1_dop_fun = tf.nn.dropout(fcon_1, keep_prob = keep_prob)

#outlayer
with tf.name_scope('OutLayer'):
    wei_out = weight_variable([1024, 10])
    bia_out = bias_variable([10])
    y_outlay = tf.matmul(fcon_1_dop_fun, wei_out) + bia_out

#train and opt
with tf.name_scope('CossEntropy'):
    cross_entro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_outlay))
#labels = y, logits = y_outlay
    tf.summary.scalar("CossEntropy", cross_entro)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entro)
#tf.train.AdamOptimizer(1e-4???).minimize(cross_entro)
correct_pre = tf.equal(tf.argmax(y_outlay, 1), tf.argmax(y, 1))
with tf.name_scope('Accurancy'):
    acc = tf.reduce_mean(tf.cast(correct_pre, tf.float32))
    tf.summary.scalar("Accurancy", acc)

#init
init = tf.global_variables_initializer()
sess.run(init)

#visualization
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('TensorBoard/', graph = tf.get_default_graph())

#opt  
for i in range(20000):
    batch = mnist.train.next_batch(50)
#batch[1], batch[0] == batch_xs, batch_ys
    if i%100 == 0:
        train_acc = acc.eval(feed_dict = {x: batch[0], y: batch[1], keep_prob: 1.0})
        print(i, train_acc)
        summary = sess.run(merged, feed_dict = {x: batch[0], y: batch[1], keep_prob: 1.0})
        writer.add_summary(summary, i)
#write.add_summary(summary, i???)
        writer.flush()
    train_step.run(feed_dict = {x: batch[0], y: batch[1], keep_prob: 0.5})
#train_step.run again???

#test
print(acc.eval(feed_dict = {x: x_test, y: y_test, keep_prob: 1.0}))

sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.12


InvalidArgumentError: You must feed a value for placeholder tensor 'Input/Placeholder' with dtype float and shape [?,784]
	 [[Node: Input/Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Input/Placeholder', defined at:
  File "/Users/ting/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ting/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ting/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ting/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ting/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/ting/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py",